In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from src.predictionModule.LoadupSamples import LoadupSamples
from src.predictionModule.MachineModels import MachineModels 

import numpy as np
import datetime
import pandas as pd
import polars as pl
import optuna

import logging
formatted_date = datetime.datetime.now().strftime("%d%b%y_%H%M").lower()

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(asctime)s - %(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]

#Output File handler
formatted_str = f"notebook-lstm-optuna-{formatted_date}"
file_handler = logging.FileHandler(f"{formatted_str}.log", mode="w")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Usage
logger.info("This will print to the notebook's output cell")

2025-08-21 11:36:19,453 - This will print to the notebook's output cell


c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import numpy as np
import torch
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [3]:
params_default = {
    "idxAfterPrediction": 5,
    'timesteps': 90,
    'target_option': 'last',
    "LoadupSamples_time_scaling_stretch": True,
    "LoadupSamples_time_inc_factor": 10,
    
    "LSTM_units": 32,
    "LSTM_num_layers": 3,
    "LSTM_dropout": 0.001,
    "LSTM_recurrent_dropout": 0.001,
    "LSTM_learning_rate": 0.001,
    "LSTM_optimizer": "adam",
    "LSTM_bidirectional": True,
    "LSTM_batch_size": 2**12,
    "LSTM_epochs": 10,
    "LSTM_l1": 0.001,
    "LSTM_l2": 0.001,
    "LSTM_inter_dropout": 0.001,
    "LSTM_input_gaussian_noise": 0.001,
    "LSTM_conv1d": True,
    "LSTM_conv1d_kernel_size": 3,
    "LSTM_loss": "mse",
    "LSTM_transformer_before": False,
    "LSTM_transformer_after": False,
    "LSTM_tf_d_model": 0,
    "LSTM_tf_nhead": 4,
    "LSTM_tf_num_layers_before": 1,
    "LSTM_tf_num_layers_after": 1,
    "LSTM_tf_dim_feedforward": 128,
    "LSTM_tf_dropout": 0.1,
    "LSTM_tf_positional_encoding": True,
    "LSTM_tf_pool": "last",
}


In [4]:
import torch
import numpy as np
from src.predictionModule.MachineModels import MachineModels
X_sm = np.random.rand(2,5,3).astype(np.float32)
y_sm = np.random.rand(2).astype(np.float32)
for before in [False, True]:
    for after in [False, True]:
        params = {**params_default, 'timesteps':5, 'LSTM_epochs':1, 'LSTM_units':4, 'LSTM_transformer_before': before, 'LSTM_transformer_after': after}
        mm = MachineModels(params)
        model,_ = mm.run_LSTM_transformer_torch(X_sm, y_sm, X_sm, y_sm, device='cpu')
        preds = mm.predict_LSTM_transformer_torch(model, X_sm, device='cpu')
        print(before, after, preds.shape)


2025-08-21 11:36:19,491 - Adjusted tf_d_model to 4 to be divisible by nhead=4


Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-21 11:36:20,552 - Epoch 1/1 — Train RMSE: 0.2346 — Validation RMSE: 0.2332


Epochs: 100%|██████████| 1/1 [00:00<00:00, 13.95it/s]

False False (2,)
2025-08-21 11:36:20,563 - Adjusted tf_d_model to 4 to be divisible by nhead=4


...

True False (2,)
2025-08-21 11:36:20,663 - Adjusted tf_d_model to 4 to be divisible by nhead=4



Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

2025-08-21 11:36:20,695 - Epoch 1/1 — Train RMSE: 0.3187 — Validation RMSE: 0.2194


Epochs: 100%|██████████| 1/1 [00:00<00:00,  5.17it/s]

True True (2,)


In [6]:
stock_group = "group_regOHLCV_over5years"
eval_date = datetime.date(year=2025, month=6, day=13)
split_date = datetime.date(year=2023, month=12, day=31)

studytime = 60*60*1
studyname = f"sandbox_lstm_optuna_{formatted_str}"
n_startup_trials = 15

In [ ]:
def objective(trial: optuna.Trial) -> float:
    q_full = 0.998
    opt_params = params_default.copy()
    opt_params["year_start"] = trial.suggest_int("year_start", 2019, 2023)
    opt_params["idxAfterPrediction"] = trial.suggest_int("idxAfterPrediction", 1, 5, step=1)
    opt_params["LoadupSamples_time_inc_factor"] = trial.suggest_int("LoadupSamples_time_inc_factor", 11, 71, step=5)
    opt_params["timesteps"] = trial.suggest_int("timesteps", 30, 90, step=5)
    opt_params["LSTM_units"] = 16
    opt_params["LSTM_num_layers"] = 1
    opt_params["LSTM_learning_rate"] = trial.suggest_float("LSTM_learning_rate", 1e-5, 1e-2, log=True)
    opt_params["LSTM_epochs"] = 2
    opt_params["LSTM_l1"] = trial.suggest_float("LSTM_l1", 1e-5, 1e-3, log=True)
    opt_params["LSTM_l2"] = trial.suggest_float("LSTM_l2", 1e-4, 1e-1, log=True)
    opt_params["LSTM_dropout"] = trial.suggest_float("LSTM_dropout", 1e-3, 1e-1, log=True)
    opt_params["LSTM_inter_dropout"] = trial.suggest_float("LSTM_inter_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_recurrent_dropout"] = trial.suggest_float("LSTM_recurrent_dropout", 1e-4, 1e-1, log=True)
    opt_params["LSTM_conv1d_kernel_size"] = 3
    opt_params["LSTM_transformer_before"] = trial.suggest_categorical("LSTM_transformer_before", [False, True])
    opt_params["LSTM_transformer_after"] = trial.suggest_categorical("LSTM_transformer_after", [False, True])
    opt_params["LSTM_tf_nhead"] = trial.suggest_categorical("LSTM_tf_nhead", [2,4,8])
    opt_params["LSTM_tf_num_layers_before"] = trial.suggest_int("LSTM_tf_num_layers_before", 0, 2)
    opt_params["LSTM_tf_num_layers_after"] = trial.suggest_int("LSTM_tf_num_layers_after", 0, 2)
    opt_params["LSTM_tf_dim_feedforward"] = trial.suggest_int("LSTM_tf_dim_feedforward", 64, 512)
    opt_params["LSTM_tf_dropout"] = trial.suggest_float("LSTM_tf_dropout", 0.0, 0.3)
    opt_params["is_single_feature"] = trial.suggest_categorical("is_single_feature", [True, False])

    ls = LoadupSamples(
        train_start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        test_dates=[eval_date],
        treegroup=None,
        timegroup=stock_group,
        params=opt_params,
    )
    ls.load_samples(main_path = "../src/featureAlchemy/bin/")

    ls.split_dataset(
        start_date=datetime.date(year=opt_params["year_start"], month=1, day=1),
        last_train_date=split_date,
        last_test_date=eval_date,
    )

    Xtrain = ls.train_Xtime
    ytrain = ls.train_ytime
    Xtest = ls.test_Xtime
    ytest = ls.test_ytime

    true_res = ls.meta_pl_test

    Xtrain = Xtrain[:, -opt_params["timesteps"]:, :]
    Xtest = Xtest[:, -opt_params["timesteps"]:, :]

    if opt_params["is_single_feature"]:
        Xtrain = Xtrain[:, :, [0]]
        Xtest = Xtest[:, :, [0]]

    mm = MachineModels(opt_params)

    starttime0 = datetime.datetime.now()
    model_lstm0, res_dict0 = mm.run_LSTM_transformer_torch(Xtrain, ytrain, Xtest, ytest, device="cuda")
    preds_train0 = mm.predict_LSTM_transformer_torch(model_lstm0, Xtrain, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    preds_test0 = mm.predict_LSTM_transformer_torch(model_lstm0, Xtest, batch_size=opt_params["LSTM_batch_size"], device="cuda")
    endtime0 = datetime.datetime.now()

    # final mask
    q2 = q_full
    mask_pred2_test_above = (preds_test0 >= np.quantile(preds_test0, q2))
    mask_pred2_test_below = (preds_test0 <= np.quantile(preds_test0, 1-q2))

    true_res_masked_above = true_res.filter(pl.Series(mask_pred2_test_above))
    true_res_masked_below = true_res.filter(pl.Series(mask_pred2_test_below))

    score = (np.mean(true_res_masked_above['target_ratio'].to_numpy())) ** (1/opt_params["idxAfterPrediction"])

    # Log some results
    def quant_dis_in_mask(mask: np.ndarray, q: float) -> float:
        if not mask.any():
            return len(mask)
        return np.quantile(np.abs(np.diff(np.where(mask)[0])), q)

    fullmask_above = mask_pred2_test_above.copy()
    fullmask_below = mask_pred2_test_below.copy()

    logger.info(f"Trial {trial.number} with params: {opt_params}")
    logger.info(f"  q2: {q2:.4f}")
    logger.info(f"  Duration0: {endtime0 - starttime0}")
    logger.info(f"  Val RMSE0 adjusted: {res_dict0['val_rmse']/opt_params['LoadupSamples_time_inc_factor']:.4f}")
    logger.info(f"  Mean all prediction: {np.mean(true_res['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean above prediction: {np.mean(true_res_masked_above['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Mean below prediction: {np.mean(true_res_masked_below['target_ratio'].to_numpy()):.4f}")
    logger.info(f"  Quantile 0.99 for distance in mask above: {quant_dis_in_mask(fullmask_above, 0.99)}")
    logger.info(f"  Quantile 0.99 for distance in mask below: {quant_dis_in_mask(fullmask_below, 0.99)}")
    logger.info(f"  Ratio for quantile-distance-to-length above: {quant_dis_in_mask(fullmask_above, 0.99) / len(fullmask_above):.4f}")
    logger.info(f"  Ratio for quantile-distance-to-length below: {quant_dis_in_mask(fullmask_below, 0.99) / len(fullmask_below):.4f}")
    logger.info(f"  Score: {score:.4f}")

    return score

optuna.logging.enable_propagation()
sampler = optuna.samplers.TPESampler(n_startup_trials=n_startup_trials)
study = optuna.create_study(
    study_name = studyname,
    storage="sqlite:///sandbox_optuna.db",
    direction="maximize",
    load_if_exists=True,
    sampler=sampler,
)
study.optimize(objective, timeout=studytime)

logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

df: pd.DataFrame = study.trials_dataframe()
logger.info("\nTrials DataFrame:")
logger.info(df.sort_values("value").to_string())

param_importances = optuna.importance.get_param_importances(study)
logger.info("Parameter Importances:")
for key, value in param_importances.items():
    logger.info(f"{key}: {value}")

[I 2025-08-21 11:39:59,044] Using an existing study with name 'sandbox_lstm_optuna_notebook-lstm-optuna-21aug25_1136' instead of creating a new one.


2025-08-21 11:39:59,044 - Using an existing study with name 'sandbox_lstm_optuna_notebook-lstm-optuna-21aug25_1136' instead of creating a new one.
2025-08-21 11:40:03,454 - Non-finite/too-large values in train y time: 36 samples.
2025-08-21 11:40:03,454 - Removing 36 samples from training data.
2025-08-21 11:40:06,524 - Adjusted tf_d_model to 2 to be divisible by nhead=2


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

2025-08-21 11:40:30,072 - Epoch 1/2 — Train RMSE: 0.5423 — Validation RMSE: 0.5350


Epochs:  50%|█████     | 1/2 [00:23<00:23, 23.53s/it]

...

[W 2025-08-21 11:41:08,686] Trial 2 failed with parameters: {'year_start': 2023, 'idxAfterPrediction': 3, 'LoadupSamples_time_inc_factor': 51, 'timesteps': 45, 'LSTM_learning_rate': 1.872421745333295e-05, 'LSTM_l1': 2.1384345827652245e-05, 'LSTM_l2': 0.005723198103939538, 'LSTM_dropout': 0.07225250852113131, 'LSTM_inter_dropout': 0.0025540900315483453, 'LSTM_recurrent_dropout': 0.00030785676539684303, 'LSTM_transformer_before': True, 'LSTM_transformer_after': True, 'LSTM_tf_nhead': 2, 'LSTM_tf_num_layers_before': 1, 'LSTM_tf_num_layers_after': 1, 'LSTM_tf_dim_feedforward': 378, 'LSTM_tf_dropout': 0.2209326943598554, 'is_single_feature': True} because of the following error: AttributeError("'DataFrame' object has no attribute 'any'").
Traceback (most recent call last):
  File "c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kimer\AppData\L

2025-08-21 11:41:08,686 - Trial 2 failed with parameters: {'year_start': 2023, 'idxAfterPrediction': 3, 'LoadupSamples_time_inc_factor': 51, 'timesteps': 45, 'LSTM_learning_rate': 1.872421745333295e-05, 'LSTM_l1': 2.1384345827652245e-05, 'LSTM_l2': 0.005723198103939538, 'LSTM_dropout': 0.07225250852113131, 'LSTM_inter_dropout': 0.0025540900315483453, 'LSTM_recurrent_dropout': 0.00030785676539684303, 'LSTM_transformer_before': True, 'LSTM_transformer_after': True, 'LSTM_tf_nhead': 2, 'LSTM_tf_num_layers_before': 1, 'LSTM_tf_num_layers_after': 1, 'LSTM_tf_dim_feedforward': 378, 'LSTM_tf_dropout': 0.2209326943598554, 'is_single_feature': True} because of the following error: AttributeError("'DataFrame' object has no attribute 'any'").
Traceback (most recent call last):
  File "c:\Users\kimer\Desktop\RandomOdyssey\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kimer\AppData\Loc

[W 2025-08-21 11:41:08,688] Trial 2 failed with value None.


2025-08-21 11:41:08,688 - Trial 2 failed with value None.


AttributeError: 'DataFrame' object has no attribute 'any'

In [ ]:
df.to_parquet(f"sandbox_lstm_optuna_{formatted_str}.parquet", index=False)